In [3]:
import os
import glob 
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

## Download Go_emotions

In [8]:
def download_go_emotions(dataframe_path):
  # 데이터셋 로드
  dataset = load_dataset("google-research-datasets/go_emotions")
  dataset_train = dataset['train'].to_pandas()   
  dataset_test = dataset['test'].to_pandas()
  dataset_validation = dataset['validation'].to_pandas()
  
  dataset = pd.concat([dataset_train, dataset_test, dataset_validation], axis=0, ignore_index=True)
  print("Orig Train #: ", len(dataset_train))
  print("Orig Test  #:", len(dataset_test))
  print("Orig Valid #:", len(dataset_validation))
  print("Orig Total #:", len(dataset))
  #dataset.head()
  
  # 6가지 감정만 추출
  only_ekman=True
  if only_ekman :  
    wanted_emotions = ['sadness', 'joy', 'fear', 'anger', 'surprise', 'disgust']
    dataset_emotions = ['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
                        'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
                        'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism',
                        'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']
    base_emotion_indices = [dataset_emotions.index(emotion) for emotion in wanted_emotions]
    print("base_emotion_indices", base_emotion_indices)
    
    selected_samples = []
    for i in range(len(dataset)):
        df_tmp = dataset.iloc[i]
        if len(df_tmp['labels']) == 1 and df_tmp['labels'][0] in base_emotion_indices:
            selected_samples.append(df_tmp)

    dataset = pd.DataFrame(selected_samples, columns=['text', 'labels', 'id']).reset_index(drop=True)
    print("New Total #:", len(dataset))
    
    dataset.to_pickle(dataframe_path)
    return dataset

In [9]:
dataframe_path = '../datasets/pkl/go_emotions.pkl'
download_go_emotions(dataframe_path)

Orig Train #:  43410
Orig Test  #: 5427
Orig Valid #: 5426
Orig Total #: 54263
base_emotion_indices [25, 17, 14, 2, 26, 11]
New Total #: 5410


,text,labels,id
0,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj
1,To make her feel threatened,[14],ed7ypvh
2,OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...,[26],edvnz26
3,Fucking coward.,[2],edk0z9k
4,Stupidly stubborn / stubbornly stupid,[2],edkh6qo
...,...,...,...
5405,I can't believe this team beat Winnipeg last w...,[26],eeblypq
5406,I would be afraid he would come back and someh...,[14],edew673
5407,It happens so much that it really scares me tbh.,[14],ee2lna8
5408,Threats and promotion of violence,[14],ef3gvik


## Download Yelp

In [1]:

def download_yelp(dataframe_path):
    #dataset already downloaded from https://github.com/shentianxiao/language-style-transfer/tree/master/data/yelp
    
    orig_path = "../datasets/original/yelp/"
    yelp_dict = {
        'sentiment.dev.0': ["dev", 0],
        'sentiment.dev.1': ["dev", 1],
        'sentiment.test.0': ["test", 0],
        'sentiment.test.1': ["test", 1],
        'sentiment.train.0': ["train", 0],
        'sentiment.train.1': ["train", 1]
    }
    labels = []
    dataset = []
    sentences = []
    files = [os.path.join(orig_path, f) for f in os.listdir(orig_path) if os.path.isfile(os.path.join(orig_path, f))]
    
    for elem in tqdm(files, desc="Preparing dataset files and saving as pkl"): 
        file = os.path.basename(elem)
        if not "reference" in file:
            with open(elem, 'r') as f:
                for l in f:
                    l = l.strip()
                    labels.append(yelp_dict[file][1])
                    dataset.append(yelp_dict[file][0])
                    sentences.append(l)

    df = pd.DataFrame({"dataset": dataset, "sentiment": labels, "sample": sentences})
    df = df.drop_duplicates(subset=["sample"], keep="first")  # sentences should be unique
    df.to_pickle(dataframe_path)
    return df

In [4]:
dataframe_path = '../datasets/pkl/yelp.pkl'
download_yelp(dataframe_path)

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/original/yelp/'